In [1]:
import re
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np

/mnt/efs/home/iet5740/Projects/UKBPPP/torch_venv/lib64/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
npx = pd.read_csv('combined_pheno_forconsortium_v1_NPX.tsv',sep='\t',index_col=0,low_memory=False)
missingRatioProt = npx.apply(lambda x: x.isna().sum()/npx.shape[0],axis=0)
npx = npx.loc[:,list(missingRatioProt[missingRatioProt < .1].index)]
missingRatioSamp = npx.apply(lambda x: x.isna().sum()/npx.shape[1],axis=1)
npx = npx.loc[list(missingRatioSamp[missingRatioSamp < .2].index),:]
#npx = npx.dropna()

info = pd.read_csv('sampleInfo.csv',index_col=0)
info = info.loc[:,map(lambda x: re.search(r'in urine',x)==None,info.columns)]
info['Sex'] = info.apply(lambda x: 0 if x['Sex']=='F' else 1 if x['Sex']=='M' else np.nan,axis=1)
print(info.shape)
df = pd.concat([npx,info],join='inner',axis=1)
df

(52363, 63)


,GLO1:Q04760:OID20107:v1:Cardiometabolic,PAG1:Q9NWQ8:OID20108:v1:Cardiometabolic,ADAM15:Q13444:OID20109:v1:Cardiometabolic,USP8:P40818:OID20110:v1:Cardiometabolic,BMP6:P22004:OID20111:v1:Cardiometabolic,ITGB1BP2:Q9UKP3:OID20112:v1:Cardiometabolic,CTSH:P09668:OID20113:v1:Cardiometabolic,BAG6:P46379:OID20114:v1:Cardiometabolic,MSTN:O14793:OID20115:v1:Cardiometabolic,BOC:Q9BWV1:OID20116:v1:Cardiometabolic,...,Oestradiol,Phosphate,Rheumatoid factor,SHBG,Total bilirubin,Testosterone,Total protein,Triglycerides,Urate,Vitamin D
eid,,,,,,,,,,,,,,,,,,,,,
5763561,-0.5539,-0.61835,-0.33685,-0.8604,0.2837,-0.82665,0.7754,-0.2148,1.2040,0.25545,...,NaN,1.326,NaN,76.58,14.96,13.185,67.83,0.770,254.3,45.3
1541419,-0.8117,-0.51535,-0.34215,-0.8946,-0.5164,-1.29915,-0.7953,-0.6748,0.5846,-0.29475,...,NaN,1.512,NaN,19.77,14.16,10.815,75.16,1.917,445.9,40.9
2845293,0.1584,0.42485,0.77525,1.5851,1.2331,1.12505,0.5892,0.5778,0.7071,0.68725,...,NaN,1.084,NaN,39.74,6.25,9.194,66.33,3.844,396.7,64.7
2178814,-0.6318,-0.10435,-0.06085,-1.1274,-0.4320,-1.67025,-0.1314,-0.5383,0.3361,0.10205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5084631,0.1961,-0.28665,0.81625,-0.1619,-0.1686,0.06795,0.5508,0.0556,0.5347,0.07815,...,NaN,1.280,NaN,41.39,10.05,14.113,73.15,3.248,300.7,65.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.206,17.3,50.28,10.10,1.694,75.97,1.643,332.1,35.4
2829755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.137,NaN,28.87,6.94,0.385,78.00,1.816,314.8,31.1
6007204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.826,NaN,46.94,9.75,1.410,70.33,0.725,281.3,49.3


In [3]:
torch.tensor(df.iloc[-2,:])

tensor([nan, nan, nan,  ..., nan, nan, nan], dtype=torch.float64)

In [4]:
torch.nan_to_num(torch.tensor(df.iloc[-2,:]))

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)

In [5]:
class proteomicsDataset(Dataset):
    def __init__(self,exprMat):
        self.exprMat = exprMat.iloc[:,:-info.shape[1]].astype(float)
        self.labels = exprMat.iloc[:,-info.shape[1]:].astype(float)
        
    def __len__(self):
        return(self.labels.shape[0])
    
    def __getitem__(self,idx):
        #exprVector = np.array(self.exprMat.iloc[idx,:])
        #label = np.array(self.labels.iloc[idx,:])
        exprVector = torch.tensor(self.exprMat.iloc[idx,:], dtype=torch.float64)
        label = torch.tensor(self.labels.iloc[idx,:], dtype=torch.float64)
        return exprVector,label

In [6]:
dataset = proteomicsDataset(df)
print(len(dataset))

49583


In [7]:
dataset[200][0].shape

torch.Size([2931])

In [8]:
dataset[200][1].shape

torch.Size([63])

In [9]:
# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
INPUT_DIM = 2931
Z_DIM = 10
H_DIM = 200
NUM_EPOCHS = 10
BATCH_SIZE = 80
LR_RATE = 3e-4

cpu


In [10]:
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

In [252]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, input_dim, z_dim, h_dim=200):
        super().__init__()
        # encoder
        self.img_2hid = nn.Linear(input_dim, h_dim)

        # one for mu and one for stds, note how we only output
        # diagonal values of covariance matrix. Here we assume
        # the pixels are conditionally independent 
        self.hid_2mu = nn.Linear(h_dim, z_dim)
        self.hid_2sigma = nn.Linear(h_dim, z_dim)

        # decoder
        self.z_2hid = nn.Linear(z_dim, h_dim)
        self.hid_2img = nn.Linear(h_dim, input_dim)
        
        self.double()

    def encode(self, x):
        h = F.relu(self.img_2hid(x))
        mu = self.hid_2mu(h)
        sigma = self.hid_2sigma(h)
        return mu, sigma

    def decode(self, z):
        new_h = F.relu(self.z_2hid(z))
        x = self.hid_2img(new_h)
        return x

    def forward(self, x):
        mu, sigma = self.encode(x)
        sigma = torch.exp(sigma)

        # Sample from latent distribution from encoder
        epsilon = torch.randn_like(sigma)
        z_reparametrized = mu + sigma*epsilon

        x = self.decode(z_reparametrized)
        return x, mu, sigma

In [253]:
# Define train function
def train(num_epochs, model, optimizer, loss_fn):
    # Start training
    for epoch in range(num_epochs):
        loop = tqdm(enumerate(train_loader))
        for i, (x, y) in loop:
            # Forward pass
            x = x.to(device).view(-1, INPUT_DIM)
            nan_in_x = torch.isnan(x)
            x = torch.nan_to_num(x)
            x_reconst, mu, sigma = model(x)

            # loss, formulas from https://www.youtube.com/watch?v=igP03FXZqgo&t=2182s
            reconst_loss = loss_fn(x_reconst[~nan_in_x], x[~nan_in_x])
            kl_div = - torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))

            # Backprop and optimize
            loss = reconst_loss + kl_div
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())

In [261]:
class VariationalAutoEncoder2(nn.Module):
    def __init__(self, input_dim, z_dim, h_dim=200):
        super().__init__()
        # encoder
        self.img_2hid = nn.Linear(input_dim, h_dim)

        # one for mu and one for stds, note how we only output
        # diagonal values of covariance matrix. Here we assume
        # the pixels are conditionally independent 
        self.hid_2mu = nn.Linear(h_dim, z_dim)
        self.hid_2sigma = nn.Linear(h_dim, z_dim)

        # decoder
        self.z_2hid = nn.Linear(z_dim, h_dim)
        self.hid_2img = nn.Linear(h_dim, input_dim)
        
        # decoder2 (for computing extra terms involving sigma)
        self.z_2hid2 = nn.Linear(z_dim, h_dim, bias=False)
        self.hid_2img2 = nn.Linear(h_dim, input_dim, bias=False)
        del self.z_2hid2.weight
        del self.hid_2img2.weight
        
        self.double()

    def encode(self, x):
        h = F.relu(self.img_2hid(x))
        mu = self.hid_2mu(h)
        sigma = self.hid_2sigma(h)
        return mu, sigma

    def decode(self, z):
        new_h = self.z_2hid(z)
        x = self.hid_2img(new_h)
        return x
    
    def decode2(self, z):
        z = torch.diag_embed(z.pow(.5))
        new_h = self.z_2hid2(z)
        x = self.hid_2img2(new_h)
        x_t = torch.transpose(x,-2,-1)
        out = torch.diagonal(torch.matmul(x_t,x),dim1=-2,dim2=-1)
        return out
    
    def extra(self, sigma):
        CA = torch.matmul(self.hid_2img.weight,self.z_2hid.weight)
        cov = torch.diag_embed(sigma)
        add_on = torch.diagonal(torch.matmul(torch.matmul(CA,cov),CA.transpose(0,1)),dim1=-2,dim2=-1)
        #add_on = torch.diagonal(torch.matmul(torch.matmul(torch.matmul(self.hid_2img.weight,self.z_2hid.weight),torch.diag_embed(sigma)),torch.matmul(self.z_2hid.weight.transpose(0,1),self.hid_2img.weight.transpose(0,1))),dim1=-2,dim2=-1)
        return add_on

    def forward(self, x):
        mu, sigma = self.encode(x)
        sigma = torch.exp(sigma)

        # Compute expected reconstructed
        #parameters = {}
        #for name, param in model.named_parameters():
        #    parameters.update({name: param})

        #CA = torch.matmul(parameters['hid_2img.weight'],parameters['z_2hid.weight'])
        #x_reconst = torch.matmul(CA,mu) + torch.matmul(parameters['hid_2img.weight'],parameters['z_2hid.bias']) + parameters['hid_2img.bias']
        x_reconst = self.decode(mu)
        add_on = self.extra(sigma)
        
        #self.z_2hid2.weight = self.z_2hid.weight
        #self.hid_2img2.weight = self.hid_2img.weight
        #add_on = self.decode2(sigma)
        
        return x_reconst, add_on, mu, sigma

In [262]:
# Define train function
def train2(num_epochs, model, optimizer, loss_fn):
    # Start training
    for epoch in range(num_epochs):
        loop = tqdm(enumerate(train_loader))
        for i, (x, y) in loop:
            # Forward pass
            x = x.to(device).view(-1, INPUT_DIM)
            nan_in_x = torch.isnan(x)
            x = torch.nan_to_num(x)
            x_reconst, add_on, mu, sigma = model(x)
            
            # loss, formulas from https://www.youtube.com/watch?v=igP03FXZqgo&t=2182s
            reconst_loss = loss_fn(x_reconst[~nan_in_x], x[~nan_in_x]) + add_on[~nan_in_x].sum()
            kl_div = - torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))

            # Backprop and optimize
            loss = reconst_loss + kl_div
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())

In [263]:
# Initialize model, optimizer, loss
model = VariationalAutoEncoder2(INPUT_DIM, Z_DIM, H_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR_RATE)
loss_fn = nn.MSELoss(reduction="sum")

In [255]:
# Run training
train(NUM_EPOCHS, model, optimizer, loss_fn)

620it [00:41, 14.86it/s, loss=5.16e+4]
620it [00:41, 14.97it/s, loss=5.14e+4]
620it [00:41, 14.87it/s, loss=5.08e+4]
620it [00:41, 14.91it/s, loss=5.21e+4]
620it [00:41, 15.01it/s, loss=5.13e+4]
620it [00:41, 14.94it/s, loss=4.7e+4] 
620it [00:41, 14.87it/s, loss=5.08e+4]
620it [00:41, 14.95it/s, loss=4.85e+4]
620it [00:41, 14.92it/s, loss=4.95e+4]
620it [00:41, 14.99it/s, loss=4.81e+4]


In [264]:
# Run training
train2(NUM_EPOCHS, model, optimizer, loss_fn)

0it [00:00, ?it/s]


RuntimeError: multi_dot(): tensor 1 must be 2D but got 3D

In [15]:
print(model)

VariationalAutoEncoder(
  (img_2hid): Linear(in_features=2931, out_features=200, bias=True)
  (hid_2mu): Linear(in_features=200, out_features=10, bias=True)
  (hid_2sigma): Linear(in_features=200, out_features=10, bias=True)
  (z_2hid): Linear(in_features=10, out_features=200, bias=True)
  (hid_2img): Linear(in_features=200, out_features=2931, bias=True)
)


In [18]:
for name, param in model.named_parameters():
    print(name)
    print(param.shape)

img_2hid.weight
torch.Size([200, 2931])
img_2hid.bias
torch.Size([200])
hid_2mu.weight
torch.Size([10, 200])
hid_2mu.bias
torch.Size([10])
hid_2sigma.weight
torch.Size([10, 200])
hid_2sigma.bias
torch.Size([10])
z_2hid.weight
torch.Size([200, 10])
z_2hid.bias
torch.Size([200])
hid_2img.weight
torch.Size([2931, 200])
hid_2img.bias
torch.Size([2931])


In [147]:
for param in model.parameters():
    print(param.shape)

torch.Size([200, 2931])
torch.Size([200])
torch.Size([10, 200])
torch.Size([10])
torch.Size([10, 200])
torch.Size([10])
torch.Size([200, 10])
torch.Size([200])
torch.Size([2931, 200])
torch.Size([2931])


In [152]:
model.hid_2img.weight.shape

torch.Size([2931, 200])

In [256]:
# Get Mu's and Sigma's for each image
mus = []
sigmas = []
for vector,label in dataset:
    vector = torch.nan_to_num(vector)
    with torch.no_grad():
        mu, sigma = model.encode(vector)
        mus.append(mu)
        sigmas.append(torch.exp(sigma))

In [257]:
mus[200]

tensor([ 0.0034, -0.0073, -1.3274, -0.8317,  0.9710,  1.1051, -1.0959, -0.4644,
        -0.3014, -1.3846], dtype=torch.float64)

In [258]:
rows = []
labels = []
for i,(vector,label) in enumerate(dataset):
    #labels.append(pd.Series(np.array(label),name=df.index[i]))
    rows.append(pd.Series(np.array(mus[i]),name=df.index[i]))

embedding = pd.concat(rows,join='inner',axis=1).transpose()
embedding.columns = ['latent'+str(i) for i in embedding.columns]
#info = pd.concat(labels,join='inner',axis=1).transpose()
print(embedding.shape)
#print(info.shape)
res = pd.concat([embedding,df.iloc[:,-info.shape[1]:]],join='inner',axis=1)
print(res)

(49583, 10)
          latent0   latent1   latent2   latent3   latent4   latent5   latent6  \
5763561 -0.680685  0.868693 -0.511292  1.306832 -0.564018  2.076067 -0.247620   
1541419  0.525212 -0.727346 -1.211630 -0.851299  0.618964 -0.533227 -0.073089   
2845293  0.720450  0.880395  1.367510  0.946002 -0.801622  0.764267  2.010155   
2178814 -0.515310 -0.852440 -1.732958  0.197488 -0.528880 -0.177469 -0.348126   
5084631 -0.129506 -0.538367  0.065576 -0.333725  0.820139  0.648899 -0.371953   
...           ...       ...       ...       ...       ...       ...       ...   
3310145 -0.364070 -0.498492 -0.386034 -0.723786 -0.828824  0.919842  0.442109   
2829755 -0.118993  0.081587  0.484513 -0.570566 -0.455666 -1.412791  0.466189   
6007204 -0.796650 -0.189002  0.189047  0.560751 -0.848744  0.369756  0.385306   
1077460  0.661683  0.726889 -0.503904 -1.065581 -0.375922  0.578437  0.202448   
5781248 -0.386401  0.356152 -0.553205 -0.448852  0.792897 -1.219424 -0.705761   

          laten

In [259]:
embedding.iloc[200,:]

latent0    0.003381
latent1   -0.007317
latent2   -1.327411
latent3   -0.831713
latent4    0.970995
latent5    1.105116
latent6   -1.095856
latent7   -0.464361
latent8   -0.301365
latent9   -1.384595
Name: 2555809, dtype: float64

In [260]:
embedding.to_csv('VAE_embeddings.csv')